In [ ]:
import pandas as pd
from os.path import join
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_sec_1 = pd.read_csv(join("./accelerometer_data_section_1.csv"))
data_sec_2 = pd.read_csv(join("./accelerometer_data_section_2.csv"))
data_sec_3 = pd.read_csv(join("./accelerometer_data_section_3.csv"))
data_sec_4 = pd.read_csv(join("./accelerometer_data_section_4.csv"))
data_sec_5 = pd.read_csv(join("./accelerometer_data_section_5.csv"))
data_sec_6 = pd.read_csv(join("./accelerometer_data_section_6.csv"))
data_sec_7 = pd.read_csv(join("./accelerometer_data_section_7.csv"))

vehicle_timestamps = pd.read_csv(join("./vehicle_timestamps.csv"))

In [ ]:
data = data_sec_1.append([data_sec_2,data_sec_3,data_sec_4,data_sec_5,data_sec_6,data_sec_7])